In [12]:
import sys
sys.path.append("/home/sagemaker-user/memoMAE/")

import torch
from tqdm import tqdm
from Model.memoMAE import memoMAE
from Experiments.utils import load_backbone_from_ckpt
from Experiments.dataloader import ImagenetData

In [2]:
device = 2
config_path = '/home/sagemaker-user/memoMAE/Configs/config_mae.yaml'
ckpt_path = '/home/sagemaker-user/memoMAE/checkpoints/mae-ViT-B-Patch-16-MemoCap400000-NumSim10-NosimEpochs500/last.ckpt'

In [3]:
mae_baseline = load_backbone_from_ckpt(config_path=config_path,
                                       ModelClass=memoMAE,
                                       ckpt_path=ckpt_path)

Loaded memoMAE. Missing keys: []
Unexpected keys: []


In [4]:
mae_baseline = mae_baseline.to(2)

In [5]:
data = ImagenetData(train_txt=None, 
                          val_txt='/home/sagemaker-user/memoMAE/Data/val.txt', 
                          root_dir='/home/sagemaker-user/memoMAE/Data/',
                          batch_size=64)

In [6]:
val_loader = data.val_dataloader()

In [13]:
LATENTS = []
LABELS = []
with torch.no_grad():
    for images, labels in tqdm(val_loader):
        latents = mae_baseline.forward_encoder_memo(images.to(2), 0., 0, memorize=False)[0]
        LATENTS.append(latents.cpu())
        LABELS.append(labels.cpu())

  2%|▏         | 9/547 [00:04<04:33,  1.97it/s]


KeyboardInterrupt: 